# NBEATSx Account Balance Forecasting - Next 30 Days

This notebook implements a complete forecasting system to predict account balance for the next 30 days using NBEATSx neural network model with optimized hyperparameters.

## Project Overview

- **Model**: NBEATSx (Neural Basis Expansion Analysis for Time Series Forecasting)
- **Objective**: Predict account balance for the next 30 days
- **Data**: Preprocessed training dataset with engineered features
- **Optimization**: Optuna hyperparameter tuning with 11-hour timeout
- **Database**: SQLite for Optuna study storage
- **Features**: Time-based features, lags, rolling statistics

## Workflow
1. **Data Import** - Load preprocessed dataset
2. **Data Validation** - Verify data quality and completeness
3. **Hyperparameter Tuning** - Optimize model parameters using Optuna
4. **Model Training** - Train final model with best parameters
5. **Forecasting** - Generate 30-day predictions with uncertainty intervals
6. **Evaluation & Analysis** - Analyze results and model performance

## 1. Import Libraries and Setup

Import all required libraries for data processing, modeling, visualization, and hyperparameter optimization.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Time series and forecasting
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import DistributionLoss

# Metrics and evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Hyperparameter optimization
import optuna
from optuna.trial import Trial
import sqlite3

# Date and time
from datetime import datetime, timedelta
import json

# Set random seeds for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

print("✅ All libraries imported successfully!")
print(f"📅 Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Optuna version: {optuna.__version__}")
print("🚀 Ready for forecasting!")

## 2. Data Loading and Validation

Load the preprocessed dataset and validate data quality. The dataset should already be preprocessed with features engineered.

In [ ]:
# Load the preprocessed dataset
print("📂 Loading preprocessed dataset...")
df = pd.read_excel("processed_train_dataset.xlsx")

print(f"✅ Dataset loaded successfully!")
print(f"📊 Dataset shape: {df.shape}")
print(f"📅 Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"📈 Total days: {len(df)}")

# Display basic info
print("\n📋 Dataset Info:")
print(f"Columns: {list(df.columns)}")
print(f"Data types:\n{df.dtypes}")

# Display first few rows
print("\n🔍 First 5 rows:")
print(df.head())

# Check for any missing values
print(f"\n❌ Missing values:")
missing_values = df.isnull().sum()
for col, missing in missing_values.items():
    if missing > 0:
        print(f"   {col}: {missing} ({missing/len(df)*100:.2f}%)")
    
if missing_values.sum() == 0:
    print("   ✅ No missing values found!")
else:
    print(f"   ⚠️ Total missing values: {missing_values.sum()}")

In [ ]:
# Data validation and feature identification
print("🔍 Data Validation and Feature Analysis")
print("="*60)

# Validate essential columns
required_columns = ['Date', 'Normalized_Balance']
missing_required = [col for col in required_columns if col not in df.columns]

if missing_required:
    print(f"❌ Missing required columns: {missing_required}")
    raise ValueError(f"Dataset must contain columns: {required_columns}")
else:
    print("✅ All required columns present")

# Identify feature types
feature_categories = {
    'date_column': 'Date',
    'target_column': 'Normalized_Balance',
    'future_features': [],
    'historical_features': []
}

# Categorize features automatically
for col in df.columns:
    if col in required_columns:
        continue
    elif any(x in col.lower() for x in ['dayofweek_sin', 'dayofweek_cos', 'sin', 'cos']):
        feature_categories['future_features'].append(col)
    elif any(x in col.lower() for x in ['ago', 'lag', 'rolling', 'mean', 'std', 'changed']):
        feature_categories['historical_features'].append(col)

print(f"\n📊 Feature Categories:")
print(f"🎯 Target: {feature_categories['target_column']}")
print(f"🔮 Future features ({len(feature_categories['future_features'])}): {feature_categories['future_features']}")
print(f"📈 Historical features ({len(feature_categories['historical_features'])}): {feature_categories['historical_features']}")

# Validate data quality
print(f"\n🔍 Data Quality Checks:")

# Check date continuity
df_sorted = df.sort_values('Date').reset_index(drop=True)
date_diff = df_sorted['Date'].diff().dt.days
missing_dates = (date_diff > 1).sum()
print(f"📅 Date continuity: {len(df_sorted) - missing_dates - 1}/{len(df_sorted) - 1} consecutive days")

# Check target variable
target_stats = df[feature_categories['target_column']].describe()
print(f"💰 Target variable stats:")
print(f"   Range: {target_stats['min']:.4f} to {target_stats['max']:.4f}")
print(f"   Mean: {target_stats['mean']:.4f}, Std: {target_stats['std']:.4f}")

# Check for outliers (values beyond 3 standard deviations)
target_col = feature_categories['target_column']
mean_val = df[target_col].mean()
std_val = df[target_col].std()
outliers = ((df[target_col] - mean_val).abs() > 3 * std_val).sum()
print(f"⚠️ Potential outliers (>3σ): {outliers} ({outliers/len(df)*100:.2f}%)")

print("\n✅ Data validation completed!")

## 3. Hyperparameter Tuning with Optuna

Optimize NBEATSx model hyperparameters using Optuna with SQLite storage and 11-hour timeout.

In [ ]:
# Setup Optuna study with SQLite storage
print("🔧 Setting up Optuna hyperparameter optimization")
print("="*60)

# Configuration
HORIZON = 30  # Forecast horizon (days)
TIMEOUT_HOURS = 11  # 11 hours timeout
TIMEOUT_SECONDS = TIMEOUT_HOURS * 3600
STUDY_NAME = "nbeats_balance_forecasting"
DB_URL = f"sqlite:///optuna_study_{STUDY_NAME}.db"

print(f"🎯 Forecast horizon: {HORIZON} days")
print(f"⏰ Timeout: {TIMEOUT_HOURS} hours ({TIMEOUT_SECONDS} seconds)")
print(f"💾 Database: {DB_URL}")

# Create or load existing study
try:
    study = optuna.create_study(
        study_name=STUDY_NAME,
        storage=DB_URL,
        direction='minimize',
        load_if_exists=True
    )
    print(f"✅ Study loaded/created: {len(study.trials)} existing trials")
except Exception as e:
    print(f"⚠️ Creating new study: {e}")
    study = optuna.create_study(
        study_name=STUDY_NAME,
        storage=DB_URL,
        direction='minimize'
    )

# Split data for hyperparameter tuning (use last 20% for validation)
VALIDATION_SIZE = 0.2
split_idx = int(len(df) * (1 - VALIDATION_SIZE))
train_data = df.iloc[:split_idx].copy()
val_data = df.iloc[split_idx:].copy()

print(f"📊 Training data: {len(train_data)} days ({df['Date'].iloc[0]} to {df['Date'].iloc[split_idx-1]})")
print(f"📊 Validation data: {len(val_data)} days ({df['Date'].iloc[split_idx]} to {df['Date'].iloc[-1]})")

# Prepare data for NeuralForecast format
def prepare_neural_forecast_data(data, feature_categories):
    """Convert data to NeuralForecast format"""
    nf_data = data.copy()
    nf_data['unique_id'] = 'balance'
    nf_data = nf_data.rename(columns={
        feature_categories['date_column']: 'ds', 
        feature_categories['target_column']: 'y'
    })
    return nf_data

train_nf = prepare_neural_forecast_data(train_data, feature_categories)
val_nf = prepare_neural_forecast_data(val_data, feature_categories)

print("✅ Data prepared for optimization!")

In [ ]:
# Future features creation function
def create_future_features(last_date, horizon, feature_categories):
    """
    Create future features for forecasting period.
    Only creates features that can be known in advance (like time-based features).
    """
    # Generate future dates
    future_dates = pd.date_range(
        start=last_date + pd.Timedelta(days=1), 
        periods=horizon, 
        freq='D'
    )
    
    # Create future dataframe
    future_df = pd.DataFrame({
        'ds': future_dates,
        'unique_id': 'balance'
    })
    
    # Add time-based features (these can be known in advance)
    future_df['dayofweek_sin'] = np.sin(2 * np.pi * future_df['ds'].dt.dayofweek / 7)
    future_df['dayofweek_cos'] = np.cos(2 * np.pi * future_df['ds'].dt.dayofweek / 7)
    
    # Add any other future features that exist in the dataset
    for feature in feature_categories['future_features']:
        if feature not in future_df.columns:
            if 'sin' in feature.lower():
                # Handle additional sine features
                if 'month' in feature.lower():
                    future_df[feature] = np.sin(2 * np.pi * future_df['ds'].dt.month / 12)
                elif 'dayofyear' in feature.lower():
                    future_df[feature] = np.sin(2 * np.pi * future_df['ds'].dt.dayofyear / 365.25)
                else:
                    future_df[feature] = 0  # Default value
            elif 'cos' in feature.lower():
                # Handle additional cosine features
                if 'month' in feature.lower():
                    future_df[feature] = np.cos(2 * np.pi * future_df['ds'].dt.month / 12)
                elif 'dayofyear' in feature.lower():
                    future_df[feature] = np.cos(2 * np.pi * future_df['ds'].dt.dayofyear / 365.25)
                else:
                    future_df[feature] = 0  # Default value
            else:
                future_df[feature] = 0  # Default for unknown future features
    
    print(f"🔮 Created future features for {horizon} days")
    print(f"📅 Future period: {future_dates[0]} to {future_dates[-1]}")
    
    return future_df

# Test future features creation
last_training_date = train_nf['ds'].max()
test_future_df = create_future_features(last_training_date, 5, feature_categories)
print("\n📋 Sample future features:")
print(test_future_df.head())

In [ ]:
# Optuna objective function
def objective(trial):
    """
    Optuna objective function for NBEATSx hyperparameter optimization.
    Returns RMSE score to minimize.
    """
    try:
        # Hyperparameter search space
        params = {
            'input_size': trial.suggest_int('input_size', 60, 200),
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
            'max_steps': trial.suggest_int('max_steps', 500, 2000),
            'batch_size': trial.suggest_int('batch_size', 16, 64),
            'n_harmonics': trial.suggest_int('n_harmonics', 1, 5),
            'n_polynomials': trial.suggest_int('n_polynomials', 1, 5),
            'dropout_prob_theta': trial.suggest_float('dropout_prob_theta', 0.0, 0.3),
            'weight_decay': trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True),
            'early_stop_patience_steps': trial.suggest_int('early_stop_patience_steps', 20, 100)
        }
        
        # N-blocks for each stack (identity, trend, seasonality)
        n_blocks = [
            trial.suggest_int('n_blocks_identity', 2, 6),
            trial.suggest_int('n_blocks_trend', 2, 6), 
            trial.suggest_int('n_blocks_seasonality', 2, 6)
        ]
        
        # Create NBEATSx model
        model = NBEATSx(
            h=HORIZON,
            input_size=params['input_size'],
            futr_exog_list=feature_categories['future_features'],
            hist_exog_list=feature_categories['historical_features'],
            
            # Architecture parameters
            stack_types=['identity', 'trend', 'seasonality'],
            n_blocks=n_blocks,
            n_harmonics=params['n_harmonics'],
            n_polynomials=params['n_polynomials'],
            
            # Training parameters
            learning_rate=params['learning_rate'],
            max_steps=params['max_steps'],
            batch_size=params['batch_size'],
            dropout_prob_theta=params['dropout_prob_theta'],
            weight_decay=params['weight_decay'],
            early_stop_patience_steps=params['early_stop_patience_steps'],
            
            # Other settings
            random_seed=42,
            scaler_type='standard',
            loss=DistributionLoss(distribution='Normal', level=[80, 90, 95]),
            
            # Reduce verbosity for optimization
            trainer_kwargs={
                'enable_progress_bar': False,
                'enable_model_summary': False
            }
        )
        
        # Create forecaster
        forecaster = NeuralForecast(models=[model], freq='D')
        
        # Fit model on training data
        forecaster.fit(df=train_nf)
        
        # Create future features for validation period
        future_df = create_future_features(
            last_date=train_nf['ds'].max(),
            horizon=len(val_nf),
            feature_categories=feature_categories
        )
        
        # Generate predictions
        forecast_df = forecaster.predict(futr_df=future_df)
        
        # Calculate RMSE
        actual = val_nf['y'].values
        predicted = forecast_df['NBEATSx'].values
        
        # Handle any potential NaN values
        mask = ~(np.isnan(actual) | np.isnan(predicted))
        if mask.sum() == 0:
            return float('inf')
            
        rmse = np.sqrt(mean_squared_error(actual[mask], predicted[mask]))
        
        # Log trial info
        trial_info = {
            'trial_number': trial.number,
            'rmse': rmse,
            'params': params,
            'n_blocks': n_blocks
        }
        
        print(f"Trial {trial.number}: RMSE = {rmse:.6f}")
        
        return rmse
        
    except Exception as e:
        print(f"Trial {trial.number} failed: {str(e)}")
        return float('inf')

print("🎯 Objective function defined!")
print("⚙️ Hyperparameter search space:")
print("   - input_size: 60-200")
print("   - learning_rate: 1e-4 to 1e-2 (log scale)")
print("   - max_steps: 500-2000")
print("   - batch_size: 16-64")
print("   - n_blocks per stack: 2-6 each")
print("   - n_harmonics: 1-5")
print("   - n_polynomials: 1-5")
print("   - dropout_prob_theta: 0.0-0.3")
print("   - weight_decay: 1e-6 to 1e-3 (log scale)")
print("   - early_stop_patience_steps: 20-100")

In [ ]:
# Run hyperparameter optimization
print("🚀 Starting hyperparameter optimization...")
print("="*60)
print(f"⏰ Timeout: {TIMEOUT_HOURS} hours")
print(f"💾 Results will be saved to: {DB_URL}")
print(f"🔄 Existing trials: {len(study.trials)}")

# Record start time
start_time = datetime.now()
print(f"🕐 Start time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# Run optimization with timeout
try:
    study.optimize(
        objective, 
        timeout=TIMEOUT_SECONDS,
        n_jobs=1,  # Single job to avoid conflicts
        show_progress_bar=True
    )
    
    optimization_completed = True
    print("\n✅ Optimization completed successfully!")
    
except KeyboardInterrupt:
    print("\n⚠️ Optimization interrupted by user")
    optimization_completed = False
    
except Exception as e:
    print(f"\n❌ Optimization failed: {e}")
    optimization_completed = False

# Record end time and duration
end_time = datetime.now()
duration = end_time - start_time
print(f"🕐 End time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Duration: {duration}")

# Display results
print("\n" + "="*60)
print("📊 OPTIMIZATION RESULTS")
print("="*60)

if len(study.trials) > 0:
    print(f"🔄 Total trials completed: {len(study.trials)}")
    print(f"🏆 Best RMSE: {study.best_value:.6f}")
    
    # Get best parameters
    best_params = study.best_params.copy()
    
    # Reconstruct n_blocks array
    n_blocks_best = [
        best_params.pop('n_blocks_identity'),
        best_params.pop('n_blocks_trend'),
        best_params.pop('n_blocks_seasonality')
    ]
    best_params['n_blocks'] = n_blocks_best
    
    print(f"\n🎯 Best hyperparameters:")
    for param, value in best_params.items():
        print(f"   {param}: {value}")
    
    # Save best parameters
    with open('best_hyperparameters.json', 'w') as f:
        json.dump(best_params, f, indent=2, default=str)
    print(f"\n💾 Best parameters saved to: best_hyperparameters.json")
    
else:
    print("❌ No trials completed")
    best_params = None

print("="*60)

In [ ]:
# Analyze optimization results
if len(study.trials) > 0:
    print("📈 OPTIMIZATION ANALYSIS")
    print("="*60)
    
    # Create trials dataframe for analysis
    trials_df = study.trials_dataframe()
    completed_trials = trials_df[trials_df['state'] == 'COMPLETE']
    
    if len(completed_trials) > 0:
        print(f"✅ Completed trials: {len(completed_trials)}")
        print(f"❌ Failed trials: {len(trials_df) - len(completed_trials)}")
        print(f"📊 Success rate: {len(completed_trials)/len(trials_df)*100:.1f}%")
        
        # Statistics
        rmse_stats = completed_trials['value'].describe()
        print(f"\n📊 RMSE Statistics:")
        print(f"   Best (min): {rmse_stats['min']:.6f}")
        print(f"   Worst (max): {rmse_stats['max']:.6f}")
        print(f"   Mean: {rmse_stats['mean']:.6f}")
        print(f"   Std: {rmse_stats['std']:.6f}")
        print(f"   Median: {rmse_stats['50%']:.6f}")
        
        # Plot optimization history
        plt.figure(figsize=(15, 10))
        
        # Plot 1: Optimization history
        plt.subplot(2, 2, 1)
        plt.plot(completed_trials['number'], completed_trials['value'], 'b-', alpha=0.7)
        plt.axhline(y=study.best_value, color='r', linestyle='--', label=f'Best RMSE: {study.best_value:.6f}')
        plt.xlabel('Trial Number')
        plt.ylabel('RMSE')
        plt.title('Optimization History')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Plot 2: RMSE distribution
        plt.subplot(2, 2, 2)
        plt.hist(completed_trials['value'], bins=20, alpha=0.7, edgecolor='black')
        plt.axvline(x=study.best_value, color='r', linestyle='--', label=f'Best: {study.best_value:.6f}')
        plt.xlabel('RMSE')
        plt.ylabel('Frequency')
        plt.title('RMSE Distribution')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Plot 3: Parameter importance (if enough trials)
        if len(completed_trials) >= 10:
            plt.subplot(2, 2, 3)
            try:
                importance = optuna.importance.get_param_importances(study)
                params = list(importance.keys())[:8]  # Top 8 parameters
                values = [importance[p] for p in params]
                
                plt.barh(params, values)
                plt.xlabel('Importance')
                plt.title('Parameter Importance')
                plt.gca().invert_yaxis()
            except:
                plt.text(0.5, 0.5, 'Parameter importance\nnot available', 
                        ha='center', va='center', transform=plt.gca().transAxes)
                plt.title('Parameter Importance')
        
        # Plot 4: Learning curve of best trial
        plt.subplot(2, 2, 4)
        # This would show learning curve if we had access to training history
        plt.text(0.5, 0.5, f'Best Trial: #{study.best_trial.number}\nRMSE: {study.best_value:.6f}', 
                ha='center', va='center', transform=plt.gca().transAxes,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
        plt.title('Best Trial Summary')
        
        plt.tight_layout()
        plt.show()
        
        # Save optimization report
        optimization_report = {
            'study_name': STUDY_NAME,
            'total_trials': len(study.trials),
            'completed_trials': len(completed_trials),
            'best_rmse': study.best_value,
            'best_params': best_params,
            'optimization_duration': str(duration),
            'rmse_statistics': rmse_stats.to_dict()
        }
        
        with open('optimization_report.json', 'w') as f:
            json.dump(optimization_report, f, indent=2, default=str)
        
        print(f"\n💾 Optimization report saved to: optimization_report.json")
    else:
        print("❌ No completed trials to analyze")
else:
    print("❌ No trials to analyze")
    best_params = None

## 4. Final Model Training

Train the final NBEATSx model using the best hyperparameters found during optimization, using the entire dataset.

In [ ]:
# Train final model with best parameters
print("🚀 Training Final Model")
print("="*60)

# Ensure we have best parameters
if best_params is None:
    print("⚠️ No optimized parameters found, using default parameters")
    best_params = {
        'input_size': 120,
        'learning_rate': 0.001,
        'max_steps': 1000,
        'batch_size': 32,
        'n_blocks': [3, 3, 3],
        'n_harmonics': 2,
        'n_polynomials': 2,
        'dropout_prob_theta': 0.1,
        'weight_decay': 1e-4,
        'early_stop_patience_steps': 50
    }

print(f"🎯 Using parameters:")
for param, value in best_params.items():
    print(f"   {param}: {value}")

# Prepare full dataset for training
full_nf_data = prepare_neural_forecast_data(df, feature_categories)
print(f"\n📊 Full training dataset: {len(full_nf_data)} days")
print(f"📅 Training period: {full_nf_data['ds'].min()} to {full_nf_data['ds'].max()}")

# Create final model with best parameters
final_model = NBEATSx(
    h=HORIZON,
    input_size=best_params['input_size'],
    futr_exog_list=feature_categories['future_features'],
    hist_exog_list=feature_categories['historical_features'],
    
    # Architecture parameters
    stack_types=['identity', 'trend', 'seasonality'],
    n_blocks=best_params['n_blocks'],
    n_harmonics=best_params['n_harmonics'],
    n_polynomials=best_params['n_polynomials'],
    
    # Training parameters
    learning_rate=best_params['learning_rate'],
    max_steps=best_params['max_steps'],
    batch_size=best_params['batch_size'],
    dropout_prob_theta=best_params.get('dropout_prob_theta', 0.0),
    weight_decay=best_params.get('weight_decay', 1e-5),
    early_stop_patience_steps=best_params.get('early_stop_patience_steps', 50),
    
    # Other settings
    random_seed=42,
    scaler_type='standard',
    loss=DistributionLoss(distribution='Normal', level=[80, 90, 95]),
    
    # Enable progress tracking for final training
    trainer_kwargs={
        'enable_progress_bar': True,
        'enable_model_summary': True
    }
)

# Create final forecaster
final_forecaster = NeuralForecast(models=[final_model], freq='D')

# Train the final model
print(f"\n🎓 Training final model...")
training_start = datetime.now()

try:
    final_forecaster.fit(df=full_nf_data)
    training_success = True
    print("✅ Final model training completed successfully!")
    
except Exception as e:
    training_success = False
    print(f"❌ Final model training failed: {e}")

training_end = datetime.now()
training_duration = training_end - training_start
print(f"⏱️ Training duration: {training_duration}")

if training_success:
    print("\n🎉 Final model ready for forecasting!")

## 5. 30-Day Balance Forecasting

Generate predictions for the next 30 days with uncertainty intervals and confidence levels.

In [ ]:
# Generate 30-day forecast
if training_success:
    print("🔮 Generating 30-Day Balance Forecast")
    print("="*60)
    
    # Create future features for the next 30 days
    last_date = full_nf_data['ds'].max()
    future_features_df = create_future_features(
        last_date=last_date,
        horizon=HORIZON,
        feature_categories=feature_categories
    )
    
    print(f"📅 Forecast period: {future_features_df['ds'].min()} to {future_features_df['ds'].max()}")
    
    # Generate forecast with uncertainty intervals
    forecast_start = datetime.now()
    
    try:
        forecast_df = final_forecaster.predict(futr_df=future_features_df)
        forecasting_success = True
        print("✅ Forecast generated successfully!")
        
    except Exception as e:
        forecasting_success = False
        print(f"❌ Forecasting failed: {e}")
        forecast_df = None
    
    forecast_end = datetime.now()
    forecast_duration = forecast_end - forecast_start
    print(f"⏱️ Forecasting duration: {forecast_duration}")
    
    if forecasting_success and forecast_df is not None:
        # Process forecast results
        forecast_df['Date'] = future_features_df['ds']
        forecast_df = forecast_df.reset_index(drop=True)
        
        # Extract predictions and confidence intervals
        point_forecast = forecast_df['NBEATSx'].values
        
        # Extract confidence intervals if available
        ci_columns = [col for col in forecast_df.columns if 'NBEATSx' in col and any(level in col for level in ['80', '90', '95'])]
        
        print(f"\n📊 Forecast Summary:")
        print(f"   Horizon: {HORIZON} days")
        print(f"   Point forecasts: {len(point_forecast)} values")
        print(f"   Confidence intervals: {len(ci_columns)} levels")
        print(f"   Available intervals: {[col.split('-')[-1] for col in ci_columns if 'hi' in col]}")
        
        # Create comprehensive forecast dataframe
        forecast_summary = pd.DataFrame({
            'Date': future_features_df['ds'],
            'Day': range(1, HORIZON + 1),
            'Predicted_Balance': point_forecast
        })
        
        # Add confidence intervals
        for col in ci_columns:
            if 'lo' in col:
                level = col.split('-')[-1]
                forecast_summary[f'Lower_CI_{level}'] = forecast_df[col]
            elif 'hi' in col:
                level = col.split('-')[-1]
                forecast_summary[f'Upper_CI_{level}'] = forecast_df[col]
        
        # Add trend information
        forecast_summary['Daily_Change'] = forecast_summary['Predicted_Balance'].diff()
        forecast_summary['Cumulative_Change'] = forecast_summary['Predicted_Balance'] - forecast_summary['Predicted_Balance'].iloc[0]
        forecast_summary['Weekly_Change'] = forecast_summary['Predicted_Balance'].diff(7)
        
        print(f"\n📈 Forecast Statistics:")
        print(f"   Starting balance: {point_forecast[0]:.4f}")
        print(f"   Ending balance: {point_forecast[-1]:.4f}")
        print(f"   Total change: {point_forecast[-1] - point_forecast[0]:.4f}")
        print(f"   Average daily change: {forecast_summary['Daily_Change'].mean():.4f}")
        print(f"   Max daily change: {forecast_summary['Daily_Change'].max():.4f}")
        print(f"   Min daily change: {forecast_summary['Daily_Change'].min():.4f}")
        
        # Display first few predictions
        print(f"\n🔍 First 10 predictions:")
        print(forecast_summary[['Date', 'Day', 'Predicted_Balance', 'Daily_Change']].head(10).to_string(index=False))
        
        # Save forecast results
        forecast_summary.to_csv('30_day_forecast.csv', index=False)
        forecast_summary.to_excel('30_day_forecast.xlsx', index=False)
        print(f"\n💾 Forecast saved to:")
        print(f"   - 30_day_forecast.csv")
        print(f"   - 30_day_forecast.xlsx")
        
    else:
        print("❌ Cannot proceed with analysis - forecasting failed")
        forecast_summary = None

else:
    print("❌ Cannot generate forecast - final model training failed")
    forecast_summary = None

In [ ]:
# Visualize forecast results
if forecasting_success and forecast_summary is not None:
    print("📊 Creating Forecast Visualizations")
    print("="*60)
    
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    
    # Plot 1: Historical + Forecast
    ax1 = axes[0, 0]
    
    # Show last 60 days of historical data for context
    hist_context = full_nf_data.tail(60)
    ax1.plot(hist_context['ds'], hist_context['y'], 'b-', label='Historical Balance', linewidth=2)
    ax1.plot(forecast_summary['Date'], forecast_summary['Predicted_Balance'], 'r-', 
             label='30-Day Forecast', linewidth=2, marker='o', markersize=3)
    
    # Add confidence intervals if available
    if 'Lower_CI_95' in forecast_summary.columns:
        ax1.fill_between(forecast_summary['Date'], 
                        forecast_summary['Lower_CI_95'], 
                        forecast_summary['Upper_CI_95'],
                        alpha=0.2, color='red', label='95% Confidence')
    
    if 'Lower_CI_80' in forecast_summary.columns:
        ax1.fill_between(forecast_summary['Date'], 
                        forecast_summary['Lower_CI_80'], 
                        forecast_summary['Upper_CI_80'],
                        alpha=0.3, color='orange', label='80% Confidence')
    
    ax1.axvline(x=last_date, color='green', linestyle='--', alpha=0.7, label='Forecast Start')
    ax1.set_title('Account Balance: Historical + 30-Day Forecast')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Normalized Balance')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # Plot 2: Daily Changes
    ax2 = axes[0, 1]
    ax2.plot(forecast_summary['Date'], forecast_summary['Daily_Change'], 'g-', 
             marker='o', markersize=4, label='Daily Change')
    ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    ax2.set_title('Predicted Daily Balance Changes')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Daily Change')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.tick_params(axis='x', rotation=45)
    
    # Plot 3: Cumulative Change
    ax3 = axes[1, 0]
    ax3.plot(forecast_summary['Date'], forecast_summary['Cumulative_Change'], 'purple', 
             marker='o', markersize=4, label='Cumulative Change')
    ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    ax3.set_title('Cumulative Balance Change from Day 1')
    ax3.set_xlabel('Date')
    ax3.set_ylabel('Cumulative Change')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.tick_params(axis='x', rotation=45)
    
    # Plot 4: Weekly Analysis
    ax4 = axes[1, 1]
    
    # Group by week for weekly analysis
    forecast_summary['Week'] = ((forecast_summary['Day'] - 1) // 7) + 1
    weekly_summary = forecast_summary.groupby('Week').agg({
        'Predicted_Balance': ['mean', 'min', 'max'],
        'Daily_Change': 'sum'
    }).round(4)
    
    weekly_summary.columns = ['Avg_Balance', 'Min_Balance', 'Max_Balance', 'Weekly_Change']
    weekly_summary = weekly_summary.reset_index()
    
    ax4.bar(weekly_summary['Week'], weekly_summary['Weekly_Change'], 
            alpha=0.7, color=['green' if x >= 0 else 'red' for x in weekly_summary['Weekly_Change']])
    ax4.set_title('Weekly Balance Changes')
    ax4.set_xlabel('Week')
    ax4.set_ylabel('Weekly Change')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print weekly summary
    print(f"\n📊 Weekly Forecast Summary:")
    print(weekly_summary.to_string(index=False))
    
    # Save visualization
    fig.savefig('30_day_forecast_visualization.png', dpi=300, bbox_inches='tight')
    print(f"\n💾 Visualization saved to: 30_day_forecast_visualization.png")
    
    # Risk Analysis
    print(f"\n⚠️ RISK ANALYSIS:")
    
    # Volatility analysis
    daily_volatility = forecast_summary['Daily_Change'].std()
    max_decline = forecast_summary['Daily_Change'].min()
    max_increase = forecast_summary['Daily_Change'].max()
    
    print(f"   📈 Daily volatility (std): {daily_volatility:.4f}")
    print(f"   📉 Largest predicted decline: {max_decline:.4f}")
    print(f"   📈 Largest predicted increase: {max_increase:.4f}")
    
    # Trend analysis
    if forecast_summary['Cumulative_Change'].iloc[-1] > 0:
        trend = "INCREASING"
        trend_emoji = "📈"
    else:
        trend = "DECREASING" 
        trend_emoji = "📉"
    
    print(f"   {trend_emoji} Overall 30-day trend: {trend}")
    print(f"   💰 Expected total change: {forecast_summary['Cumulative_Change'].iloc[-1]:.4f}")
    
    # Weeks with negative changes
    negative_weeks = (weekly_summary['Weekly_Change'] < 0).sum()
    print(f"   ⚠️ Weeks with declining balance: {negative_weeks}/4")

else:
    print("❌ Cannot create visualizations - forecasting data not available")

## 6. Summary and Model Performance

Final summary of the forecasting pipeline and key results.

In [ ]:
# Final Summary and Conclusions
print("🎉 NBEATS BALANCE FORECASTING SUMMARY")
print("="*70)

# Project Overview
project_summary = {
    'model_type': 'NBEATSx Neural Network',
    'forecast_horizon': f'{HORIZON} days',
    'optimization_method': 'Optuna with SQLite storage',
    'optimization_timeout': f'{TIMEOUT_HOURS} hours',
    'data_source': 'processed_train_dataset.xlsx',
    'output_files': [
        '30_day_forecast.csv',
        '30_day_forecast.xlsx', 
        '30_day_forecast_visualization.png',
        'best_hyperparameters.json',
        'optimization_report.json'
    ]
}

print("📋 PROJECT OVERVIEW:")
for key, value in project_summary.items():
    print(f"   {key}: {value}")

# Model Performance Summary
if best_params and forecasting_success:
    print(f"\n🏆 MODEL PERFORMANCE:")
    print(f"   Best validation RMSE: {study.best_value:.6f}")
    print(f"   Total optimization trials: {len(study.trials)}")
    print(f"   Successfully completed trials: {len(study.trials_dataframe()[study.trials_dataframe()['state'] == 'COMPLETE'])}")
    
    if forecast_summary is not None:
        print(f"\n📊 FORECAST CHARACTERISTICS:")
        print(f"   Forecast period: {forecast_summary['Date'].min()} to {forecast_summary['Date'].max()}")
        print(f"   Starting balance: {forecast_summary['Predicted_Balance'].iloc[0]:.4f}")
        print(f"   Ending balance: {forecast_summary['Predicted_Balance'].iloc[-1]:.4f}")
        print(f"   Total predicted change: {forecast_summary['Cumulative_Change'].iloc[-1]:.4f}")
        print(f"   Average daily volatility: {forecast_summary['Daily_Change'].std():.4f}")

# Technical Details
print(f"\n🔧 TECHNICAL DETAILS:")
print(f"   Features used: {len(feature_categories['future_features']) + len(feature_categories['historical_features'])}")
print(f"   Future features: {feature_categories['future_features']}")
print(f"   Historical features: {feature_categories['historical_features']}")
print(f"   Training data points: {len(df)}")
print(f"   Training period: {df['Date'].min()} to {df['Date'].max()}")

# Execution Times
total_execution_time = datetime.now() - start_time if 'start_time' in locals() else "Not available"
print(f"\n⏱️ EXECUTION TIMES:")
print(f"   Total execution: {total_execution_time}")
if 'duration' in locals():
    print(f"   Optimization duration: {duration}")
if 'training_duration' in locals():
    print(f"   Final training duration: {training_duration}")
if 'forecast_duration' in locals():
    print(f"   Forecasting duration: {forecast_duration}")

# Files Generated
print(f"\n💾 FILES GENERATED:")
import os
output_files = [
    '30_day_forecast.csv',
    '30_day_forecast.xlsx',
    '30_day_forecast_visualization.png',
    'best_hyperparameters.json',
    'optimization_report.json',
    f'optuna_study_{STUDY_NAME}.db'
]

for file in output_files:
    if os.path.exists(file):
        file_size = os.path.getsize(file)
        print(f"   ✅ {file} ({file_size:,} bytes)")
    else:
        print(f"   ❌ {file} (not found)")

# Next Steps and Recommendations
print(f"\n🚀 NEXT STEPS & RECOMMENDATIONS:")
print("   1. Review forecast visualization for business insights")
print("   2. Monitor actual vs predicted values to validate model performance")
print("   3. Update model weekly/monthly with new data")
print("   4. Consider ensemble methods for improved accuracy")
print("   5. Implement automated retraining pipeline")
print("   6. Set up monitoring alerts for significant forecast deviations")

# Success Status
overall_success = (
    best_params is not None and 
    training_success and 
    forecasting_success and 
    forecast_summary is not None
)

print(f"\n{'🎉' if overall_success else '⚠️'} OVERALL STATUS: {'SUCCESS' if overall_success else 'PARTIAL SUCCESS'}")

if overall_success:
    print("✅ All pipeline components completed successfully!")
    print("✅ 30-day balance forecast is ready for business use!")
else:
    print("⚠️ Some components may need attention - check logs above")

print("="*70)
print("🏁 NBEATS BALANCE FORECASTING COMPLETED")
print("="*70)